<a href="https://colab.research.google.com/github/ThakkarHarsh/Amazon-Personalize/blob/main/Amazon_Personalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
import os
import json
import numpy as np
import pandas as pd
import time
os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-1'
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
!pip install awscli

In [ ]:
!aws configure

In [ ]:
bucket = "bfor"       # replace with the name of your S3 bucket


In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

In [ ]:
s3 = boto3.resource('s3',
         aws_access_key_id='access key',
         aws_secret_access_key= 'secret key')

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv("extra2.csv", index=False)

s3.Bucket(bucket).Object( ).upload_file("extra2.csv")

In [ ]:
k = pd.read_csv("/content/User.csv")
k.to_csv("extra.csv", index=False)
s3.Bucket(bucket).Object("extra.csv").upload_file("extra.csv")

Item data uploaded directly into bucket in aws.

In [ ]:
schema = {
  "type": "record",
  "name": "Users",
  "schema":"Users",
  "namespace": "com.amazonaws.personalize.schema",
 
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "AGE",
          "type": "int"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
  ],
  "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "User-schema",
    schema = json.dumps(schema))

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

In [ ]:
schema={
  "type": "record",
  "name": "Items",
  "schema":"Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "GENRE",
          "type": "string",
          "categorical": True
      }
  ],
  "version": "1.0"
}
create_schema_response = personalize.create_schema(
    name = "Item-schema",
    schema = json.dumps(schema))

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:129962929409:schema/Item-schema",
  "ResponseMetadata": {
    "RequestId": "30f425eb-2ec7-40da-a0f7-f07c78342412",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:34:31 GMT",
      "x-amzn-requestid": "30f425eb-2ec7-40da-a0f7-f07c78342412",
      "content-length": "82",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "schema":"Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",  
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "Interaction-schema",
    schema = json.dumps(schema))

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:129962929409:schema/Interaction-schema",
  "ResponseMetadata": {
    "RequestId": "2ad86064-5e51-45da-a5d3-ecbb8c525d56",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:34:42 GMT",
      "x-amzn-requestid": "2ad86064-5e51-45da-a5d3-ecbb8c525d56",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "extra-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset-group/extra-group",
  "ResponseMetadata": {
    "RequestId": "5bd26a08-679b-423b-8933-9ebc789c9804",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:35:07 GMT",
      "x-amzn-requestid": "5bd26a08-679b-423b-8933-9ebc789c9804",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "demo-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn =  "arn:aws:personalize:ap-southeast-1:129962929409:schema/Interaction-schema"
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset/extra-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "762a3e72-fe9f-4866-823f-ab45f033ce3a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:38:49 GMT",
      "x-amzn-requestid": "762a3e72-fe9f-4866-823f-ab45f033ce3a",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type ="Items"
create_dataset_response = personalize.create_dataset(
    name = "demo-item-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = "arn:aws:personalize:ap-southeast-1:129962929409:schema/Item-schema"
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset/extra-group/ITEMS",
  "ResponseMetadata": {
    "RequestId": "56765744-7a9e-43c3-8b7b-b4254ab50292",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:39:22 GMT",
      "x-amzn-requestid": "56765744-7a9e-43c3-8b7b-b4254ab50292",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type ="Users"
create_dataset_response = personalize.create_dataset(
    name = "demo-user-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = "arn:aws:personalize:ap-southeast-1:129962929409:schema/User-schema"
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset/extra-group/USERS",
  "ResponseMetadata": {
    "RequestId": "32986323-7d29-4195-9c87-8cfe2f35023a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:39:49 GMT",
      "x-amzn-requestid": "32986323-7d29-4195-9c87-8cfe2f35023a",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Thu, 26 Dec 2019 04:43:44 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'U/AnQ8uRcuIjSyghXwGXamsOIKVWsVM/NTkPd3gTIxmKpfXtqHW/Mxcqsu+yx81y17BNRdWQ65g=',
   'x-amz-request-id': '852BAA36CF24B251'},
  'HTTPStatusCode': 204,
  'HostId': 'U/AnQ8uRcuIjSyghXwGXamsOIKVWsVM/NTkPd3gTIxmKpfXtqHW/Mxcqsu+yx81y17BNRdWQ65g=',
  'RequestId': '852BAA36CF24B251',
  'RetryAttempts': 0}}

In [ ]:
iam = boto3.client("iam")
role_name ="ExtraRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        { "Sid": "",
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::129962929409:role/ExtraRole


In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-dataset2-import-job",
    datasetArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset/extra-group/USERS",
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, "extra.csv")
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset-import-job/demo-dataset2-import-job",
  "ResponseMetadata": {
    "RequestId": "0814c8ea-49a4-428e-9749-3945e745dd0e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:50:06 GMT",
      "x-amzn-requestid": "0814c8ea-49a4-428e-9749-3945e745dd0e",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-dataset1-import-job",
    datasetArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset/extra-group/ITEMS",
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, "extra1.csv")
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset-import-job/demo-dataset1-import-job",
  "ResponseMetadata": {
    "RequestId": "51a1ba71-e8c9-4016-8c57-a5df9bbca9f9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 04:56:57 GMT",
      "x-amzn-requestid": "51a1ba71-e8c9-4016-8c57-a5df9bbca9f9",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-dataset3-import-job",
    datasetArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset/extra-group/INTERACTIONS",
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, "extra2.csv")
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:129962929409:dataset-import-job/demo-dataset3-import-job",
  "ResponseMetadata": {
    "RequestId": "301e06f0-047c-4719-87e2-91d4d31dd0fa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 05:01:37 GMT",
      "x-amzn-requestid": "301e06f0-047c-4719-87e2-91d4d31dd0fa",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset-import-job/demo-dataset1-import-job"
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset-import-job/demo-dataset2-import-job"
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset-import-job/demo-dataset3-import-job"
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [ ]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # aws-hrnn selected for demo purposes
list_recipes_response


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '1067',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 26 Dec 2019 05:37:34 GMT',
   'x-amzn-requestid': '46ace8fa-8000-40a7-9a5f-c322399f7365'},
  'HTTPStatusCode': 200,
  'RequestId': '46ace8fa-8000-40a7-9a5f-c322399f7365',
  'RetryAttempts': 0},
 'recipes': [{'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 19, 22, 57, 15, 868000, tzinfo=tzlocal()),
   'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE'},
  {'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 19, 22, 57, 15, 868000, tzinfo=tzlocal()),
   'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE'},
  {'creationDateTime': datetime.datetime(2019, 6, 10, 0,

In [ ]:
create_solution_response = personalize.create_solution(
    name = "Demo-solution",
    datasetGroupArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset-group/extra-group",
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-1:129962929409:solution/Demo-solution",
  "ResponseMetadata": {
    "RequestId": "f82c46be-cd23-42c8-bde2-64158822f587",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 05:40:52 GMT",
      "x-amzn-requestid": "f82c46be-cd23-42c8-bde2-64158822f587",
      "content-length": "88",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:129962929409:solution/Demo-solution/0ba7e84c",
  "ResponseMetadata": {
    "RequestId": "bec7e0b1-c376-4106-8c12-4aa788e3dd3e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 05:41:45 GMT",
      "x-amzn-requestid": "bec7e0b1-c376-4106-8c12-4aa788e3dd3e",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_P

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:129962929409:solution/Demo-solution/0ba7e84c",
  "metrics": {
    "coverage": 0.325,
    "mean_reciprocal_rank_at_25": 0.1844,
    "normalized_discounted_cumulative_gain_at_10": 0.1905,
    "normalized_discounted_cumulative_gain_at_25": 0.2861,
    "normalized_discounted_cumulative_gain_at_5": 0.1458,
    "precision_at_10": 0.0573,
    "precision_at_25": 0.0503,
    "precision_at_5": 0.0539
  },
  "ResponseMetadata": {
    "RequestId": "4505cbcd-1fc5-4131-b958-3edf250eafc7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 06:20:36 GMT",
      "x-amzn-requestid": "4505cbcd-1fc5-4131-b958-3edf250eafc7",
      "content-length": "401",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "Demo-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-1:129962929409:campaign/Demo-campaign",
  "ResponseMetadata": {
    "RequestId": "e11f24f7-cdce-4605-a3dc-62b145cf3cc4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 06:21:19 GMT",
      "x-amzn-requestid": "e11f24f7-cdce-4605-a3dc-62b145cf3cc4",
      "content-length": "88",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


In [ ]:
items = pd.read_csv('/content/extra.csv')
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

USER: 429
ITEM: Conversations with J.K. Rowling


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(89),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=1)))

Recommendations: [
 "The Portrait of a Lady",
 "The Ravishing of Lol Stein",
 "The Death of Ivan Ilych And Other Stories",
 "Confesiones de un chef",
 "The Coming Economic Collapse: How You Can Thrive When Oil Costs $200 a Barrel",
 "The Portable Chaucer",
 "Running Dog",
 "Paul Kirk's Championship Barbecue Sauces: 175 Make-Your-Own Sauces  Marinades  Dry Rubs  Wet Rubs  Mops and Salsas",
 "The Gettysburg Address",
 "Orestes and Other Plays",
 "Anna Karenina",
 "Lincoln at Gettysburg: The Words That Remade America",
 "Bakkhai",
 "The Known World",
 "The Compleat Works of Wllm Shkspr (abridged)",
 "Gates of Fire: An Epic Novel of the Battle of Thermopylae",
 "Ruby Ann's Down Home Trailer Park BBQin' Cookbook",
 "A Short History of Nearly Everything",
 "The John McPhee Reader (John McPhee Reader  #1)",
 "The Oresteia",
 "Oresteia",
 "The Zahir",
 "Medea",
 "Time for the Stars (Heinlein's Juveniles  #10)",
 "The Bacchae of Euripides: A Communion Rite"
]


In [ ]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-sims" # aws-hrnn selected for demo purposes
list_recipes_response


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '1067',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 26 Dec 2019 11:37:41 GMT',
   'x-amzn-requestid': 'b6f254b8-fda8-4c95-a506-2166fe190f8c'},
  'HTTPStatusCode': 200,
  'RequestId': 'b6f254b8-fda8-4c95-a506-2166fe190f8c',
  'RetryAttempts': 0},
 'recipes': [{'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 19, 22, 57, 15, 868000, tzinfo=tzlocal()),
   'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE'},
  {'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 19, 22, 57, 15, 868000, tzinfo=tzlocal()),
   'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE'},
  {'creationDateTime': datetime.datetime(2019, 6, 10, 0,

In [ ]:
create_solution_response = personalize.create_solution(
    name = "new-solution",
    datasetGroupArn = "arn:aws:personalize:ap-southeast-1:129962929409:dataset-group/extra-group",
    recipeArn = "arn:aws:personalize:::recipe/aws-sims"
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution",
  "ResponseMetadata": {
    "RequestId": "19506ce1-1e02-42b7-8296-95737e218031",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 11:45:12 GMT",
      "x-amzn-requestid": "19506ce1-1e02-42b7-8296-95737e218031",
      "content-length": "87",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution"
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution/74c61189",
  "ResponseMetadata": {
    "RequestId": "208aa8cb-197f-4864-a8bd-8275166a2678",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 11:45:53 GMT",
      "x-amzn-requestid": "208aa8cb-197f-4864-a8bd-8275166a2678",
      "content-length": "103",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution/74c61189"
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_P

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution/74c61189"
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution/74c61189",
  "metrics": {
    "coverage": 0.4875,
    "mean_reciprocal_rank_at_25": 0.113,
    "normalized_discounted_cumulative_gain_at_10": 0.1178,
    "normalized_discounted_cumulative_gain_at_25": 0.1885,
    "normalized_discounted_cumulative_gain_at_5": 0.0779,
    "precision_at_10": 0.0448,
    "precision_at_25": 0.0382,
    "precision_at_5": 0.0414
  },
  "ResponseMetadata": {
    "RequestId": "3cabd398-ca7b-402c-b40c-2c32e78c4c2f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 12:40:48 GMT",
      "x-amzn-requestid": "3cabd398-ca7b-402c-b40c-2c32e78c4c2f",
      "content-length": "400",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "new-campaign",
    solutionVersionArn = "arn:aws:personalize:ap-southeast-1:129962929409:solution/new-solution/74c61189",
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-1:129962929409:campaign/new-campaign",
  "ResponseMetadata": {
    "RequestId": "95a29a79-00f5-4dbc-8aab-b923e9cadb74",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 26 Dec 2019 12:41:08 GMT",
      "x-amzn-requestid": "95a29a79-00f5-4dbc-8aab-b923e9cadb74",
      "content-length": "87",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = "arn:aws:personalize:ap-southeast-1:129962929409:campaign/new-campaign"
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


In [ ]:
items = pd.read_csv('/content/books.csv')
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

USER: 588
ITEM: Gap Creek


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(89),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=1)))

Recommendations: [
 "Chapterhouse: Dune (Dune Chronicles #6)",
 "eBay Business All-in-One Desk Reference for Dummies",
 "The Long Goodbye (Philip Marlowe  #6)",
 "The Seven Habits of Highly Effective People",
 "Acharnians",
 "Questions and Answers on Death and Dying: A Companion Volume to On Death and Dying",
 "The Power of One (The Power of One  #1)",
 "The Power of One: One Person  One Rule  One Month",
 "Daily Reflections For Highly Effective People: Living the 7 Habits of Highly Successful People Every Day",
 "Truman",
 "The Histories",
 "Liar's Poker",
 "The Changeling",
 "Vida de Pi",
 "The Tragedy of Pudd'nhead Wilson/Those Extraordinary Twins",
 "eBay for Dummies",
 "The Last Dance: Encountering Death and Dying",
 "The Shadow of the Wind (The Cemetery of Forgotten Books  #1)",
 "Dune Messiah (Dune Chronicles #2)",
 "Traders  Guns & Money: Knowns and Unknowns in the Dazzling World of Derivatives",
 "We Were Not Like Other People",
 "The Last Assassin (John Rain  #5)",
 "Money to